# Pre-Requisites

In [ ]:
!pip3 install -U sentence-transformers

In [3]:
!pip3 install pinecone-client pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 KB 573.1 kB/s eta 0:00:00a 0:00:01
  Using cached pandas-1.4.4-cp38-cp38-macosx_11_0_arm64.whl (10.3 MB)
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 KB 1.6 MB/s eta 0:00:00a 0:00:01
  Using cached PyYAML-6.0-cp38-cp38-macosx_12_0_arm64.whl
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
  Using cached numpy-1.23.3-cp38-cp38-macosx_11_0_arm64.whl (13.3 MB)
  Using cached certifi-2022.9.14-py3-none-any.whl (162 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
You should consider upgrading via the '/Users/rezamohideen/gitrepos/semantic-search/ENV/bin/python3 -m pip install --upg

## Query Generation

In [5]:
import pandas as pd

query_passage_df = pd.read_csv('datasets/query_passage.csv')
query_passage_df.head()

,Unnamed: 0,query,passage
0,0,what does ocd mean,One of the most helpful things in my own recov...
1,1,is it good to talk about an ocd,One of the most helpful things in my own recov...
2,2,does ocd really help recovery,One of the most helpful things in my own recov...
3,3,what is being a positive person,There is something powerful about knowing that...
4,4,who is an inspirational person,There is something powerful about knowing that...


## Negative Mining

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256

model


In [ ]:
import pinecone

API_KEY = "55e5d3b2-c5f7-494d-8323-3c79b45150a6"

pinecone.init(
    api_key=API_KEY,
    environment='us-west1-gcp'
)
# create a new genq index if does not already exist
if 'negative-mine' not in pinecone.list_indexes():
    pinecone.create_index(
        'negative-mine',
        dimension=model.get_sentence_embedding_dimension(),
        metric='dotproduct',
        pods=1  # increase for faster mining
    )
# connect
index = pinecone.Index('negative-mine')

In [8]:
def get_pairs(df):
    pairs = []
    for index, row in df.iterrows():
        pairs.append((row['query'], row['passage']))

    return pairs

In [ ]:
pairs_gen = get_pairs()  # generator that loads (query, passage) pairs
blogs = pd.read_csv('datasets/blogs.csv')

pairs = []
to_upsert = []
passage_batch = []
id_batch = []
metadata_batch = []
batch_size = 64  # encode and upload size

for i, (query, passage) in enumerate(pairs_gen):
    pairs.append((query, passage))
    # we do this to avoid passage duplication in the vector DB
    if passage not in passage_batch: 
        passage_batch.append(passage)
        id_batch.append(str(i))

        metadata = blogs[blogs['text'] == passage]
        metadata_batch.append({'paragraph': metadata['paragraph'], 'article': metadata['article']})
    # on reaching batch_size, we encode and upsert
    if len(passage_batch) == batch_size:
        embeds = model.encode(passage_batch).tolist()
        # upload to index
        index.upsert(vectors=list(zip(id_batch, embeds, metadata_batch)))
        # refresh batches
        passage_batch = []
        id_batch = []
        metadata_batch = []
        
# check number of vectors in the index
index.describe_index_stats()